# Анализ схожести научных текстов с помощью методов естественной обработки языка и машинного обучения
## Береза Анастасия 
## Учебная группа о.ИЗДтс 23.2/Б3-22

## Установка зависимостей


In [ ]:
# При первом запуске раскомментировать строку ниже

# %pip install tensorflow kagglehub pandas numpy scikit-learn joblib tqdm psutil nltk pymorphy3 matplotlib torch

## Импорт библиотек


In [ ]:
import kagglehub
import os
import shutil

from pathlib import Path
import pandas as pd
import re
from collections import Counter
import matplotlib.pyplot as plt
from itertools import tee

from typing import Dict, List

import pymorphy3
from nltk import download
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from tqdm import tqdm

import torch
import json

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import joblib

import numpy as np
from collections import defaultdict
from sklearn.metrics.pairwise import cosine_similarity


In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Используемое устройство:", device)

Используемое устройство: cuda


In [5]:
import torch
print(torch.__version__)
print(torch.version.cuda)
print(torch.cuda.is_available())


2.9.1+cu126
12.6
True


## Загрузка датасета

In [ ]:
path = kagglehub.dataset_download("ergkerg/russian-scientific-articles")
print("Path to dataset files:", path)

# Копирование в папку проекта

destination_path = os.path.join(os.getcwd(), "russian-scientific-articles")
shutil.copytree(path, destination_path, dirs_exist_ok=True)

print("Файлы датасета перенесены в:", destination_path)

## Анализ набора данных

In [ ]:
# Сбор датасета из txt-файлов

rows = []
for root in ["russian-scientific-articles/data_3_1", "russian-scientific-articles/data_3"]:
     for p in Path(root).rglob("*.txt"):
        rows.append({
              "category": p.parent.name,
              "file": str(p),
              "text": p.read_text(encoding="utf-8", errors="ignore")
        })

df = pd.DataFrame(rows)
df.shape


In [ ]:
df.head(5)

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
df['text'].describe(include='all')

In [ ]:
df.drop_duplicates(['text',], inplace=True)
df['text'].describe(include='all')

In [ ]:
# Извлечение слов из всех статей

def raw_tokens(text):
    return re.findall(r"\w+", str(text).lower())

tokens = []
for article in df["text"].dropna():
    tokens.extend(raw_tokens(article))

# Гистограмма для топ-100 слов

top_n = 100
freq = Counter(tokens).most_common(top_n)
words, counts = zip(*freq)

plt.figure(figsize=(25,4))
plt.bar(words, counts)
plt.xticks(rotation=70, ha="right")
plt.title("Топ-100 слов")
plt.show()

print("Топ-100 слов:\n")
for i, (w, c) in enumerate(zip(words, counts), start=1):
    print(f"{i:3}. {w:<20} {c}")

# Топ пар из 2 слов 

def bigrams(seq):
    a, b = tee(seq)
    next(b, None)
    return zip(a, b)

bigram_freq = Counter(bigrams(tokens)).most_common(50)
print("\nТоп 50-биграмм")
for (w1, w2), c in bigram_freq:
    print(f"{w1} {w2}: {c}")

In [ ]:
# Стоп-слова

download("stopwords", quiet=True)
stop_ru = set(stopwords.words("russian"))
stop_en = set(stopwords.words("english"))
custom_stop = {
    # указатели, ссылки
    "doi","org","orcid","http","https","url","удк", 
    # предлоги
    "в","и","с","на","по","для","что","как","от","из","при","но","же","у","о","к",
    # предлоги на англ
    "the","of","in","on","and","a","to","for","is","are",
    # одиночные буквы
    "к","р","п","г","т","е","а","м","н","л","у",
    # цифры
    "0","1","2","3","4","5","6","7","8","9","10","11","12","13","14","15","16","17","18","19","20"
}
custom_bigram_stop = {
    # служебные
    "и в","а также","при этом","а в","на основе","в том","и др","в качестве","так и","что в","не только",
    "но и", "таким образом","том что","том числе","в рамках","в результате","в россии","в виде","в этом",
    "с помощью","на рис",
    # пары чисел
    "1 1","1 2","1 0","2 1","2 2","2 3","2 0",
    "0 0","0 1","0 5","а а","в в","в а",
}
stop_all = stop_ru | stop_en | custom_stop | custom_bigram_stop

morph = pymorphy3.MorphAnalyzer()
english_stemmer = SnowballStemmer("english")

keep_english = False
min_len = 3

# Сохранять кэш слов

lemma_cache: Dict[str, str] = {}
stem_cache: Dict[str, str] = {}

# Наборы шаблон-выражений

URL_RE = re.compile(r"https?://\S+|www\.\S+", re.IGNORECASE)
EMAIL_RE = re.compile(r"\b[\w.+-]+@[\w-]+\.[\w.-]+\b")
DOI_RE = re.compile(r"\b10\.\d{4,9}/[-._;()/:A-Za-z0-9]+\b", re.IGNORECASE)
ORCID_RE = re.compile(r"\b\d{4}-\d{4}-\d{4}-\d{3}[0-9X]\b")
NUMBER_RE = re.compile(r"\b\d[\d.,/%:-]*\d\b")
TOKEN_RE = re.compile(r"[A-Za-z\u0400-\u04FF]+")
CYRILLIC_RE = re.compile(r"[\u0400-\u04FF]")
LATIN_RE = re.compile(r"[A-Za-z]")

# Функция для очистки по шаблонам

def strip_noise(text: str) -> str:
    text = text.replace("\u00a0", " " ).replace("\ufeff", " " )
    text = URL_RE.sub(" " , text)
    text = EMAIL_RE.sub(" " , text)
    text = DOI_RE.sub(" " , text)
    text = ORCID_RE.sub(" " , text)
    text = NUMBER_RE.sub(" " , text)
    return text

# Функция предобработки текста 

def clean_text(raw: str) -> str:
    text = strip_noise(raw.lower())
    lemmas: list[str] = []
    for token in TOKEN_RE.findall(text):
        if len(token) < min_len:
            continue
        has_cyr = bool(CYRILLIC_RE.search(token))
        has_lat = bool(LATIN_RE.search(token))
        if has_cyr and has_lat:
            continue
        if has_cyr:
            lemma = lemma_cache.get(token)
            if lemma is None:
                lemma = morph.parse(token)[0].normal_form
                lemma_cache[token] = lemma
            if len(lemma) >= min_len and lemma not in stop_all:
                lemmas.append(lemma)
        elif has_lat and keep_english:
            stem = stem_cache.get(token)
            if stem is None:
                stem = english_stemmer.stem(token)
                stem_cache[token] = stem
            if len(stem) >= min_len and stem not in stop_all:
                lemmas.append(stem)
    return " ".join(lemmas)

# Предобработка датасета с помощью функции 

clean_df = (
    df.assign(clean_text=df["text"].map(clean_text))
      .loc[lambda d: d["clean_text"] != ""]
      .reset_index(drop=True)
)

clean_df.head()


In [ ]:
clean_df.info()

In [ ]:
clean_df["clean_text"].tolist()

### Визуализация частоты использования очищенных слов

In [ ]:
tokens = clean_df["clean_text"].str.split().explode()
freq = tokens.value_counts().head(30)

plt.figure(figsize=(10, 6))
freq.sort_values().plot(kind="barh")
plt.xlabel("Частота")
plt.ylabel("Слова")
plt.title("Топ-30 самых частых лемм после очистки")
plt.tight_layout()
plt.show()

In [ ]:
# Сохранение в файл очищенный набор данных
clean_df.to_json("cleaned_dataset.jsonl", orient="records", lines=True, force_ascii=False)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Количество статей по рубрикам
category_counts = (clean_df["category"].value_counts().sort_values())

# Вывод столбчатой диаграммы
fig, ax = plt.subplots(figsize=(25, 5))
ax.bar(category_counts.index.astype(str), category_counts.values)
ax.set_xlabel("Рубрика")
ax.set_ylabel("Количество статей")
ax.set_title("Распределение статей по рубрикам")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.show()


In [7]:
# Загрузка набора данных из файла
clean_df = pd.read_json("cleaned_dataset.jsonl", lines=True)
clean_df.head()


,category,file,text,clean_text
0,1 Автоматика. Вычислительная техника,russian-scientific-articles\data_3_1\1 Автомат...,"﻿2011 Компьютерная оптика, том 35, № 2 \n\nАЛГ...",компьютерный оптика алгоритм встраивание полух...
1,1 Автоматика. Вычислительная техника,russian-scientific-articles\data_3_1\1 Автомат...,﻿Software & Systems no....,программный продукт система дата подача статья...
2,1 Автоматика. Вычислительная техника,russian-scientific-articles\data_3_1\1 Автомат...,﻿Выделение контуров на изображениях с помощью ...,выделение контур изображение помощь алгоритм к...
3,1 Автоматика. Вычислительная техника,russian-scientific-articles\data_3_1\1 Автомат...,﻿Программные продукты и системы / Software & S...,программный продукт система дата подача статья...
4,1 Автоматика. Вычислительная техника,russian-scientific-articles\data_3_1\1 Автомат...,﻿Алгоритм поэтапного уточнения проективного пр...,алгоритм поэтапный уточнение проективный преоб...


# Обучение модели

## Подготовка

In [8]:
# Разделение на обучающую и валидационную выборку

train_df, test_df = train_test_split(clean_df, test_size=0.2, stratify=clean_df["category"], random_state=42)

print("Размер train:", train_df.shape)
print("Размер test:", test_df.shape)

Размер train: (1952, 4)
Размер test: (488, 4)


In [ ]:
# Векторизация TF-IDF
vectorizer = TfidfVectorizer(max_features=30000, ngram_range=(1, 2), lowercase=False, min_df=2)

train_texts = train_df["clean_text"].tolist()
test_texts = test_df["clean_text"].tolist()

X_train = vectorizer.fit_transform(train_texts)
X_test = vectorizer.transform(test_texts)

y_train = train_df["category"].astype(str).to_numpy()
y_test = test_df["category"].astype(str).to_numpy()

print("Матрица X_train:", X_train.shape)
print("Матрица X_test:", X_test.shape)

# Сохранение векторизатора для будущего использования
joblib.dump(vectorizer, "tfidf_vectorizer.joblib")

Матрица X_train: (1952, 30000)
Матрица X_test: (488, 30000)


['tfidf_vectorizer.joblib']

In [10]:
print("NNZ train:", X_train.nnz)
print("Среднее число ненулевых признаков на документ:",
      X_train.nnz / X_train.shape[0])

NNZ train: 1958084
Среднее число ненулевых признаков на документ: 1003.1168032786885


In [16]:
# Функция создания пар текстов

def make_pairs(X, y, n_pos=3, n_neg=3, random_state=42):
    rng = np.random.RandomState(random_state)
    y = np.asarray(y)

    # Индексы документов по категориям

    label2idx = defaultdict(list)
    for idx, label in enumerate(y):
        label2idx[label].append(idx)

    all_index = np.arange(len(y))
    pair_i = []
    pair_j = []
    pair_labels = []

    for label, idxs in label2idx.items():
        idxs = np.asarray(idxs)
        other_index = np.setdiff1d(all_index, idxs)

        for i in idxs:
            if len(idxs) > 1:
                pos_candidates = idxs[idxs != i]
                n_sample_pos = min(n_pos, len(pos_candidates))
                pos = rng.choice(pos_candidates, size=n_sample_pos, replace=False)
                for j in pos:
                    pair_i.append(i)
                    pair_j.append(j)
                    pair_labels.append(1)
            if len(other_index) > 0:
                n_sample_neg = min(n_neg, len(other_index))
                neg = rng.choice(other_index, size=n_sample_neg, replace=False)
                for j in neg:
                    pair_i.append(i)
                    pair_j.append(j)
                    pair_labels.append(0)

    pair_i = np.asarray(pair_i)
    pair_j = np.asarray(pair_j)
    pair_labels = np.asarray(pair_labels, dtype=int)

    print(f"Сформировано пар: {len(pair_labels)} "
          f"(положительных: {(pair_labels == 1).sum()}, отрицательных: {(pair_labels == 0).sum()})")

    return pair_i, pair_j, pair_labels

In [ ]:
# Формирование обучающих данных для модели

train_pairs = make_pairs(X_train, y_train, n_pos=3, n_neg=3, random_state=42)
test_pairs  = make_pairs(X_test,  y_test,  n_pos=3, n_neg=3, random_state=43)

train_idx_i, train_idx_j, y_train_pairs = train_pairs
test_idx_i,  test_idx_j,  y_test_pairs  = test_pairs


Сформировано пар: 11712 (положительных: 5856, отрицательных: 5856)
Сформировано пар: 2925 (положительных: 1461, отрицательных: 1464)


In [ ]:
# Вычисление сходства с помощью косинуса для каждой пары

def pair_cosine_features(X, idx_i, idx_j):
    idx_i = np.asarray(idx_i)
    idx_j = np.asarray(idx_j)
    sims = []
    for one_i, one_j in zip(idx_i, idx_j):
        Xi = X[one_i]
        Xj = X[one_j]
        cos_value = cosine_similarity(Xi, Xj)[0, 0]
        sims.append([cos_value])
    sims_array = np.array(sims)
    return sims_array

# Косинусное сходство всех пар для наборов

X_train_pairs = pair_cosine_features(X_train, train_idx_i, train_idx_j)
X_test_pairs = pair_cosine_features(X_test, test_idx_i, test_idx_j)

print("Размер X_train_pairs:", X_train_pairs.shape)
print("Размер X_test_pairs:", X_test_pairs.shape)